# 2. Upload JSON data and retrieve physical values using signal definitions

In this case, we will focus on the following methods.

- Upload JSON data to the server as string
- Register a signal definition that decomposes JSON data by key
- Retrieve data by specifying the signal definition registered above

## Scenario
Convert the CSV acquired in **"1. Retrieve time series data and save as CSV"** into JSON and register as new time series data. After that, we retrieve the number contained in the JSON.

After uploading the time stamp and JSON string pair to intdash, define a signal definition and retrieve the deserialized data.

## Preparation

Before starting this scenario, prepare the following.

- Edge for data upload
- CSV created in **"1. Retrieve time series data and save as CSV"**

### Data to be used
In this scenario, the following data needs to be prepared on the server side.

| Data item | Data name that appears in this scenario |
|:---|:---|
| Edge to register time series data |sdk_edge1|
| Time series data(CSV) (\*1)|sdk_sample.csv|
| Signal definition(\*2) | `sp_ACCX`, `sp_ACCY`, `sp_ACCZ`|

(\*1) Use the CSV obtained in "1. Retrieve time series data and save as CSV"  
(\*2) Register the JSON signal definition. For registration method, check the following steps.

### Import packages and create a client
For `url` given to ` intdash.Client`, specify the environment of the intdash server. For `edge_token`, specify the token issued for the edge you use.
(\* Login with `username` and `password` is also possible, but it is recommended to use edge token for continued use.)

In [36]:
import json
import pandas as pd

import intdash
from intdash import intdash, timeutils

# Create client

client = intdash.Client(
    url = "https://example.intdash.jp",
    edge_token = "your_token",
)

## Create JSON strings and associate with timestamps
Convert the CSV data created in **"1. Retrieve time series data and save as CSV"** into JSON format. Use a Tuple to store the converted time stamp and JSON character string.

In [12]:
df = pd.read_csv('sdk_sample.csv' , index_col=0)
df

,sp_ACCY,sp_ACCZ,sp_ACCX
time,,,
2020-07-09T08:34:11.095032000Z,0.041493,0.217996,0.048094
2020-07-09T08:34:11.115194000Z,0.123338,0.305914,0.038043
2020-07-09T08:34:11.135356000Z,0.046133,0.436974,-0.051388
2020-07-09T08:34:11.155517000Z,-0.034988,0.233272,0.102698
2020-07-09T08:34:11.175679000Z,-0.055907,-0.164569,0.047395
...,...,...,...
2020-07-09T08:34:14.361262000Z,0.506122,0.231640,-1.372483
2020-07-09T08:34:14.381424000Z,0.354834,-0.354698,-1.570656
2020-07-09T08:34:14.401586000Z,-0.216771,0.578869,-0.578583


In [34]:
# convert format : {index -> {column -> value}}
dicts = df.to_dict(orient='index')

# convert tuple 
dict_list = list(dicts.items())

In [35]:
dict_list[0]

('2020-07-09T08:34:11.095032000Z',
 {'sp_ACCY': 0.041493, 'sp_ACCZ': 0.217996, 'sp_ACCX': 0.048094})

Convert from dict type to JSON string.

In [70]:
dict_list = [(d[0], json.dumps(d[1]))  for d in dict_list]

In [71]:
dict_list[0]

('2020-07-09T08:34:11.095032000Z',
 '{"sp_ACCY": 0.041493, "sp_ACCZ": 0.217996, "sp_ACCX": 0.048094}')

## Register the JSON string associated with the time stamp on the server
After converting the time stamp and JSON string pair created above to `intdash.DataPoint`, register it on the server.

### Create a measurement
First, create measurement data to be associated with the time series data.
Retrieve the edge information.

In [37]:
edges = client.edges.list(name='sdk_edge1')
sdk_edge1 = edges[0]

In [38]:
sdk_edge1.name

'sdk_edge1'

In [40]:
meas = client.measurements.create(
    edge_uuid= sdk_edge1.uuid, 
    basetime=timeutils.str2timestamp(dict_list[0][0])  #  The start time specifies the time of  first data.
)

In [41]:
print(meas)

uuid: 82827deb-31b0-4093-aa64-28a717fbca06
name: 
description: 
edge_uuid: bcecc99c-d255-4666-b4a7-6b492fe0986a
duration: 0 days 00:00:00
ended: False
basetime: 2020-07-09 08:34:11.095032+00:00
basetime_type: manual
processed_ratio: 0
protected: False
markers: None
created_at: 2020-07-16 08:49:15.170176+00:00
updated_at: 2020-07-16 08:49:15.170176+00:00


### Add time series data to the measurement
Convert the data to `intdash.DataPoint`.

In [72]:
dps = [
    intdash.DataPoint(
        elapsed_time=timeutils.str2timestamp(d[0]) - meas.basetime,  # Time elapsed from the start of measurement.
        data_type=intdash.DataType.string.value, #  store JSON as string.
        channel=1, # fixed at 1.
        data_payload=intdash.data.String(data_id='json_data', value=d[1]).to_payload()
    ) 
    for d in dict_list
]

In [73]:
print(dps[1])

elapsed_time: 0 days 00:00:00.020162
ndata_type: 10
data: b'\tjson_data{"sp_ACCY": 0.123338, "sp_ACCZ": 0.305914, "sp_ACCX": 0.038043}'


Register the converted results as data points of the measurement.

In [74]:
client.data_points.store(
    measurement_uuid = meas.uuid,  # uuid of measurement.
    data_points = dps,
)

Confirm that the measurement is updated.

In [75]:
meas = client.measurements.get(
    uuid=meas.uuid
)
print(meas)

uuid: 82827deb-31b0-4093-aa64-28a717fbca06
name: 
description: 
edge_uuid: bcecc99c-d255-4666-b4a7-6b492fe0986a
duration: 0 days 00:00:03.346878
ended: True
basetime: 2020-07-09 08:34:11.095032+00:00
basetime_type: manual
processed_ratio: 1
protected: False
markers: None
created_at: 2020-07-16 08:49:15.170176+00:00
updated_at: 2020-07-16 11:00:33.031773+00:00


## Register the conversion definition linked to JSON

For the value corresponding to a specific key in the JSON string, set the conversion method to physical values. Use `intdash.ConversionJSON` for this setting. A label is given to the created conversion definition. See below for more information.

[create-signal-json.ipynb](../0_create-signals/create-signal-json.ipynb)

The association to be set in this scenario is shown in the table below:

| JSON key | Label to be given to the data after conversion to physical values |
|---|---|
|sp_ACCX|json_ACCX|
|sp_ACCY|json_ACCY|
|sp_ACCZ|json_ACCZ|


## Confirm the signal definition to be used with JSON
Confirm that the signal definitions are correctly registered.

In [56]:
signals = client.signals.list(label='json')
for s in signals:
    print(s.label,  end=', ')

json_ACCX, json_ACCY, json_ACCZ, 

## Retrieve time series data by specifying the signal definitions
Similar to **"1. Retrieve time series data and save as CSV"**, specify the signal definitions and retrieve the data.

In [76]:
dps = client.data_points.list(
    edge_name='sdk_edge1',
    start=timeutils.str2timestamp('2020-07-09 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-10 00:00:00+09:00'),
    labels=['json_ACCX', 'json_ACCY', 'json_ACCZ']
)

In [77]:
print(dps[0])

time: 2020-07-09T08:34:11.095032000Z
measurement_uuid: 82827deb-31b0-4093-aa64-28a717fbca06
data_type: 11
channel: 1
data_id: json_ACCX
data_payload: b'\tjson_ACCX\x94\x84D\xda\xc6\x9f\xa8?'


Convert the `data_payload` to a numerical value and confirm that you can get the value of `json_ACCX`.

In [79]:
intdash.data.Float.from_payload(dps[0].data_payload).value

0.048094

If you want to retrieve the JSON data as it is, you can retrieve it by specifying the data_id (`json_data`) that you registered.

In [59]:
dps = client.data_points.list(
    edge_name='sdk_edge1',
    start=timeutils.str2timestamp('2020-07-09 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-10 00:00:00+09:00'),
    id_queries=[intdash.IdQuery(
        data_type=intdash.DataType.string,
        data_id='json_data',
        channel=1
    )]
)

In [62]:
print(dps[0])

time: 2020-07-09T08:34:11.095032000Z
measurement_uuid: 82827deb-31b0-4093-aa64-28a717fbca06
data_type: 10
channel: 1
data_id: json_data
data_payload: b"\tjson_data{'sp_ACCY': 0.041493, 'sp_ACCZ': 0.217996, 'sp_ACCX': 0.048094}"
